In [30]:
%reset -f

In [1]:
from bs4 import BeautifulSoup
from os import path
import os
from os.path import join
import pandas as pd
from pandas import Series, DataFrame
import re
import xml.etree.ElementTree as et
import zipfile

ir21_zip_files_folder = r'C:/Users/balob/Downloads/DATA_IR21/IR21_ZIPPED'
downloads = r'C:/Users/balob/Downloads/DATA_IR21'
dmi_netpfx = 'DMI.dmi_netpfx_export_Thu_Jan_16_2020.csv'
gsma_nn_gt_file = 'Global_Title_Number_Ranges_2019-12-28.csv'
dmi_diam_mcc_mnc_file = 'DMI DIAM.dmi_diam_oper_mcc_mnc_export_Sun_Dec_29_2019.csv'

In [2]:
# Распаковка zip архивов документов IR21 в папку downloads

def unzip_file(path_to_zip_file, directory_to_extract_to):
    zip_ref = zipfile.ZipFile(path_to_zip_file, 'r')
    zip_ref.extractall(directory_to_extract_to)
    zip_ref.close()
    
def find_zip_files(path_name):
    file_names_all = os.listdir(path_name)
    files = [file for file in file_names_all if path.isfile(path.join(path_name, file)) and file.find('.zip') != -1]
    return files

zip_files = find_zip_files(ir21_zip_files_folder)
for zip_file in zip_files:
    print(zip_file)
    unzip_file(join(ir21_zip_files_folder, zip_file), downloads)

IR21_ARGCM_AMX Argentina S.A._20190422_01.zip
IR21_ARGTM_Telefonica Moviles Argentina S.A._20180907_01.zip
IR21_ARGTP_Telecom Argentina S.A._20190621_01.zip
IR21_BRACL_Claro S.A_20190307_01.zip
IR21_CIVAC_Atlantique Telecom Cote d_Ivoire_20190416_01.zip
IR21_CIVTL_MTN Cote d_Ivoire S.A_20191129150259.zip
IR21_Comium Cote d_Ivoire SA (CCI SA)_CIVCN_06022015_00.zip
IR21_G-Mobile Corporation_MNGGM_24062014_01.zip
IR21_HND02_Telefonica Celular_ S.A. _CELTEL_  _20180913_01.zip
IR21_HNDME_Servicios de Comunicaciones de Honduras S.A. de C.V._20190312_01.zip
IR21_IND28_Aircel Limited_20170922_01.zip
IR21_INDAT_Bharti Airtel Limited_20190217_01.zip
IR21_INDBO_Idea Cellular Limited_20180410_01.zip
IR21_INDBO_Vodafone Idea Limited_20190917_01.zip
IR21_INDCA_Bharat Sanchar Nigam Limited_20200114105428.zip
IR21_INDDL_Mahanagar Telephone Nigam Limited_20190108_02.zip
IR21_INDJX_Reliance Jio Infocomm Limited_20200114083413.zip
IR21_INDT0_Tata Teleservices Limited_20180309_01.zip
IR21_ISR01_Partner Co

In [3]:
# Подготовка списка файлов xml

def find_txt_files_and_folders(path_name):
    file_names_all = os.listdir(path_name)
    files = [file for file in file_names_all if path.isfile(path.join(path_name, file)) and file.find('.xml') != -1]
    return files

ir21_xml_files = find_txt_files_and_folders(downloads)
for i, file in enumerate(ir21_xml_files):
    print(f'{i:03} - {file}')

000 - IR21_ARGCM_AMX Argentina S.A._20190422_01.xml
001 - IR21_ARGTM_Telefonica Moviles Argentina S.A._20180907_01.xml
002 - IR21_ARGTP_Telecom Argentina S.A._20190621_01.xml
003 - IR21_BRACL_Claro S.A_20190307_01.xml
004 - IR21_CIVAC_Atlantique Telecom Cote d_Ivoire_20190416_01.xml
005 - IR21_CIVTL_MTN Cote d_Ivoire S.A_20191129150259.xml
006 - IR21_Comium Cote d_Ivoire SA (CCI SA)_CIVCN_06022015_00.xml
007 - IR21_CRICL_Claro CR Telecomunicaciones_ S.A. _20190312_01.xml
008 - IR21_G-Mobile Corporation_MNGGM_24062014_01.xml
009 - IR21_HND02_Telefonica Celular_ S.A. _CELTEL_  _20180913_01.xml
010 - IR21_HNDME_Servicios de Comunicaciones de Honduras S.A. de C.V._20190312_01.xml
011 - IR21_IND28_Aircel Limited_20170922_01.xml
012 - IR21_INDAT_Bharti Airtel Limited_20190217_01.xml
013 - IR21_INDBO_Idea Cellular Limited_20180410_01.xml
014 - IR21_INDBO_Vodafone Idea Limited_20190917_01.xml
015 - IR21_INDCA_Bharat Sanchar Nigam Limited_20200114105428.xml
016 - IR21_INDDL_Mahanagar Telephone 

In [4]:
# Загрузка связки TAGIG - MCCMNC в DataFrame

# Функция для поиска связки TAGIG - MCCMNC

def find_tadig_mccmnc(downloads, file_ir21):
    tree = et.parse(path.join(downloads, file_ir21))
    root = tree.getroot()
    ns = re.match(r'{.*}', root.tag).group(0)
    
    sender_tadig = root.find(f'{ns}RAEXIR21FileHeader').find(f'{ns}SenderTADIG').text
    organisation_name = root.find(f'{ns}OrganisationInfo').find(f'{ns}OrganisationName').text
    
    tadig_mccmnc_list = []
    gt_nn_list = []
    
    for network in root.iter(f'{ns}Network'):
        tadig = network.find(f'{ns}TADIGCode').text
        for mccmnc in network.iter(f'{ns}CCITT_E212_NumberSeries'):
            mccmnc = ''.join([mccmnc.find(f'{ns}MCC').text, mccmnc.find(f'{ns}MNC').text])
            tadig_mccmnc_list.extend([[organisation_name, sender_tadig, tadig, mccmnc]])
            
        for gt_nn_ranges in network.iter(f'{ns}GT_NumberRanges'):
            for gt_nn_range in gt_nn_ranges.iter(f'{ns}NumberRange'):
#                 gt_nn = ''.join([gt_nn_range.find(f'{ns}CC').text, gt_nn_range.find(f'{ns}NDC').text])
                for sn_range in gt_nn_range.iter(f'{ns}SN_Range'):
                    gt_nn_start = ''.join([gt_nn_range.find(f'{ns}CC').text, gt_nn_range.find(f'{ns}NDC').text,
                                     sn_range.find(f'{ns}SN_RangeStart').text])
                    gt_nn_end = ''.join([gt_nn_range.find(f'{ns}CC').text, gt_nn_range.find(f'{ns}NDC').text,
                                     sn_range.find(f'{ns}SN_RangeStop').text])
                    gt_nn_list.extend([[organisation_name, sender_tadig, tadig, gt_nn_start, gt_nn_end]])

    return tadig_mccmnc_list, gt_nn_list

df_tadig = DataFrame(columns = ['NETWORK_NAME', 'SENDER_TADIG', 'TADIG', 'MCCMNC'])
df_nn_ir21 = DataFrame(columns = ['NETWORK_NAME', 'SENDER_TADIG', 'TADIG', 'GT_NN_START', 'GT_NN_END'])

for file_ir21 in ir21_xml_files:
    try:
        tadig_mccmnc_list = find_tadig_mccmnc(downloads, file_ir21)
    except Exception as e:
        print(f'File {file_ir21} was nor red with the error {e}')
    else:
        df_tadig = df_tadig.append(DataFrame(tadig_mccmnc_list[0],
                                             columns = ['NETWORK_NAME', 'SENDER_TADIG', 'TADIG', 'MCCMNC']))
        df_nn_ir21 = df_nn_ir21.append(DataFrame(tadig_mccmnc_list[1],
                                             columns = ['NETWORK_NAME', 'SENDER_TADIG', 'TADIG', 'GT_NN_START', 'GT_NN_END']))

df_tadig.reset_index(drop=True, inplace=True)
df_nn_ir21.reset_index(drop=True, inplace=True)

# df_tadig.to_csv(join(downloads, 'tadig_mccmnc_ir21.csv'), index=False)
display(df_tadig.head(3))
display(df_nn_ir21.head(3))

File IR21_Comium Cote d_Ivoire SA (CCI SA)_CIVCN_06022015_00.xml was nor red with the error 'NoneType' object has no attribute 'text'
File IR21_G-Mobile Corporation_MNGGM_24062014_01.xml was nor red with the error 'NoneType' object has no attribute 'text'
File IR21_Mahanagar Telephone (Mauritius) Ltd._MUSMT_26012015_01.xml was nor red with the error 'NoneType' object has no attribute 'text'
File IR21_Salt (Liechtenstein) AG_LIEVE_04042016_01.xml was nor red with the error 'NoneType' object has no attribute 'text'
File IR21_Vodafone India Limited_INDHM_02082016_07.xml was nor red with the error 'NoneType' object has no attribute 'text'


,NETWORK_NAME,SENDER_TADIG,TADIG,MCCMNC
0,AMX Argentina S.A.,ARGCM,ARGCM,722310
1,Telefónica Móviles Argentina S.A.,ARGTM,ARGTM,72207
2,Telefónica Móviles Argentina S.A.,ARGTM,ARG01,722010


,NETWORK_NAME,SENDER_TADIG,TADIG,GT_NN_START,GT_NN_END
0,AMX Argentina S.A.,ARGCM,ARGCM,543200000000,543209999999
1,Telefónica Móviles Argentina S.A.,ARGTM,ARGTM,540790000000,540799999999
2,Telefónica Móviles Argentina S.A.,ARGTM,ARG01,540790000000,540799999999


In [5]:
# Загрузка таблицы netpfx DMI

df_netpfx=pd.read_csv(join(downloads, dmi_netpfx), dtype='object')
df_netpfx.head(3)

,Operator,Prefix,Number/Range,SSN,MCC,MNC
0,GRCCO,3097,1,7,202,02
1,GRCCO,3097,1,149,202,02
2,GRCSH,30699,1,7,202,10


In [6]:
# Загрузка таблицы NN GSMA

def find_range_nn_gsma(df):
    if df.SN_START and re.sub(r'0+$', '', df.SN_START) == re.sub(r'9+$', '', df.SN_STOP):
        return df.CCNDC + re.sub(r'0+$', '', df.SN_START)
    elif df.RANGE_START == '':
        return df.CCNDC
    
df_nn_gsma = pd.read_csv(join(downloads, gsma_nn_gt_file), dtype='str'
                    ,usecols=['Country','Operator','TADIG code','CC','NDC', 'SN range start', 'SN range stop'])
df_nn_gsma['CCNDC'] = df_nn_gsma.CC + df_nn_gsma.NDC
df_nn_gsma.fillna('', inplace=True)

df_nn_gsma.drop(['CC','NDC'], axis=1, inplace=True)
df_nn_gsma.rename({'Country':'COUNTRY','Operator':'OPERATOR','TADIG code':'TADIG'
              ,'SN range start':'SN_START','SN range stop':'SN_STOP'}, axis=1, inplace=True)

df_nn_gsma['RANGE_START'] = df_nn_gsma.apply(lambda x: '' if x.SN_START == '' else x.CCNDC + x.SN_START, axis=1)
df_nn_gsma['RANGE_STOP'] = df_nn_gsma.apply(lambda x: '' if x.SN_STOP == '' else x.CCNDC + x.SN_STOP, axis=1)

df_nn_gsma['NN_RANGE'] = df_nn_gsma.apply(find_range_nn_gsma, axis=1) 

df_nn_gsma.fillna('', inplace=True)

df_nn_gsma = df_nn_gsma[['COUNTRY','OPERATOR','TADIG','NN_RANGE','CCNDC','SN_START','SN_STOP','RANGE_START','RANGE_STOP']]

df_nn_gsma.head(3)

,COUNTRY,OPERATOR,TADIG,NN_RANGE,CCNDC,SN_START,SN_STOP,RANGE_START,RANGE_STOP
0,Afghanistan,Afghan Wireless Communication Company,AFGAW,93702,9370,2000000,2999999,93702000000,93702999999
1,Afghanistan,Emirates Telecommunications Corporation (ETISA...,AFGEA,9378,9378,,,,
2,Afghanistan,Emirates Telecommunications Corporation (ETISA...,AFGEA,9373,9373,,,,


In [7]:
# Загрузка таблицы Operator MCC MNC DMI_DIAM

df_dmi_diam_mcc_mnc=pd.read_csv(join(downloads, dmi_diam_mcc_mnc_file), dtype='object')
df_dmi_diam_mcc_mnc['MCCMNC'] = df_dmi_diam_mcc_mnc.MCC + df_dmi_diam_mcc_mnc.MNC
df_dmi_diam_mcc_mnc.drop(['MCC','MNC'], axis=1, inplace=True)
df_dmi_diam_mcc_mnc.rename({'DMI Operator':'TADIG'}, axis=1, inplace=True)
df_dmi_diam_mcc_mnc.head(3)

,TADIG,MCCMNC
0,AFGAW,41201
1,GUYGT,738002
2,GUYUM,73801


In [8]:
def prepare_toc_delete_netpfx(df):
    """df[['Prefix','Number/Range','SSN']] from NETPFX DMI table"""
    for [prefix, ssn] in df.loc[df.SSN != '1', ['Prefix','SSN']].values:
        print(f'dmi.netpfx.remove,PREFIX="{prefix}",SSN="{ssn}";')

In [16]:
tadig = 'INDA1'

print('\nMCCMNC from IR21:')
display(df_tadig[df_tadig.TADIG==tadig])

print('\nMCCMNC from DMI_DIAM:')
display(df_dmi_diam_mcc_mnc[df_dmi_diam_mcc_mnc.TADIG==tadig])

print('-'*80)

print('\nGT NN range from IR21 and GSMA:')
display(df_nn_ir21[df_nn_ir21.TADIG==tadig])
display(df_nn_gsma.loc[df_nn_gsma.TADIG == tadig])

print('\nGT NN range from DMI:')
df_temp = df_netpfx[(df_netpfx.Operator == tadig)].sort_values(by=['Number/Range','SSN','Prefix'])
display(df_temp)
print('\nTOC commands to delete from DMI:\n')
prepare_toc_delete_netpfx(df_temp)


MCCMNC from IR21:


,NETWORK_NAME,SENDER_TADIG,TADIG,MCCMNC
47,Bharti Airtel Limited,INDAT,INDA1,40492



MCCMNC from DMI_DIAM:


,TADIG,MCCMNC


--------------------------------------------------------------------------------

GT NN range from IR21 and GSMA:


,NETWORK_NAME,SENDER_TADIG,TADIG,GT_NN_START,GT_NN_END


,COUNTRY,OPERATOR,TADIG,NN_RANGE,CCNDC,SN_START,SN_STOP,RANGE_START,RANGE_STOP
843,India,Bharti Airtel Limited,INDA1,919892,919892,,,,



GT NN range from DMI:


,Operator,Prefix,Number/Range,SSN,MCC,MNC



TOC commands to delete from DMI:



In [58]:
# display(df_nn_gsma.loc[df_nn_gsma.CCNDC.str.startswith('46707', na=False)])
display(df_nn_gsma.loc[df_nn_gsma.OPERATOR == 'KT Corporation'])

,COUNTRY,OPERATOR,TADIG,NN_RANGE,CCNDC,SN_START,SN_STOP,RANGE_START,RANGE_STOP
1145,"Korea, Republic of",KT Corporation,KORKF,82102919,82102919,,,,


In [16]:
display(df_nn_ir21[df_nn_ir21.NETWORK_NAME == 'KT Corporation'])

,NETWORK_NAME,SENDER_TADIG,TADIG,GT_NN_START,GT_NN_END


In [17]:
display(df_tadig[df_tadig.NETWORK_NAME == 'Telenor Sverige AB'])

,NETWORK_NAME,SENDER_TADIG,TADIG,MCCMNC
100,Telenor Sverige AB,SWEEP,SWEEP,24008
101,Telenor Sverige AB,SWEEP,SWETG,24006
102,Telenor Sverige AB,SWEEP,SWECN,24042
103,Telenor Sverige AB,SWEEP,AAZTC,90129


,NETWORK_NAME,SENDER_TADIG,TADIG,MCCMNC


In [15]:
print('\nMCCMNC from IR21:')
# display(df_tadig[df_tadig.SENDER_TADIG.str.startswith('IND', na=False)])
display(df_tadig[df_tadig.MCCMNC.str.startswith('4049', na=False)])


MCCMNC from IR21:


,NETWORK_NAME,SENDER_TADIG,TADIG,MCCMNC
20,Aircel Limited,INDAC,IND09,40491
36,Bharti Airtel Limited,INDAT,INDA8,40493
39,Bharti Airtel Limited,INDAT,INDA7,40495
40,Bharti Airtel Limited,INDAT,INDA2,40490
41,Bharti Airtel Limited,INDAT,INDA4,40494
42,Bharti Airtel Limited,INDAT,INDA6,40497
44,Bharti Airtel Limited,INDAT,INDA3,40498
47,Bharti Airtel Limited,INDAT,INDA1,40492
49,Bharti Airtel Limited,INDAT,INDA5,40496


In [18]:
print('\nGT NN range from GSMA:')
# display(df_nn_gsma.loc[df_nn_gsma.CCNDC.str.startswith('1876', na=False)])
# display(df_nn_gsma.loc[df_nn_gsma.NN_RANGE == '504'])
display(df_nn_gsma.loc[df_nn_gsma.RANGE_START.str.startswith('167192901', na=False)])


GT NN range from GSMA:


,COUNTRY,OPERATOR,TADIG,NN_RANGE,CCNDC,SN_START,SN_STOP,RANGE_START,RANGE_STOP
673,Guam,"DOCOMO PACIFIC, INC. (Guam)",GUMDP,167192901,1671,9290100,9290199,16719290100,16719290199
675,Guam,"DOCOMO PACIFIC, INC. (Guam)",GUMHT,167192901,1671,9290100,9290199,16719290100,16719290199


In [29]:
print('\nMCCMNC from IR21:')
display(df_tadig[df_tadig.MCCMNC.str.startswith('973', na=False)])
display(df_nn_gsma.loc[df_nn_gsma.CCNDC.str.startswith('674', na=False)])


MCCMNC from IR21:


,NETWORK_NAME,SENDER_TADIG,TADIG,MCCMNC


,COUNTRY,OPERATOR,TADIG,NN_RANGE,CCNDC,SN_START,SN_STOP,RANGE_START,RANGE_STOP
590,Fiji,Digicel (Fiji) Ltd,FJIDP,,67455,61000,61125,6745561000,6745561125


### Developments

In [25]:
file_ir21 = 'IR21_CRICL_Claro CR Telecomunicaciones_ S.A. _20190312_01.xml'

with open(join(downloads,file_ir21)) as inf:
    ir21 = inf.read()

xml_ir21 = BeautifulSoup(ir21,'xml')

msisdn_list = []
for MSISDN_NumberRanges in xml_ir21.findAll('MSISDN_NumberRanges'):
    for MSISDN_RangeData in MSISDN_NumberRanges.findAll('MSISDN_RangeData'):
        for NumberRange in  MSISDN_RangeData.findAll('NumberRange'):
            msisdn_list.append([NumberRange.CC.text + NumberRange.NDC.text, 'msisdn'])

nn_list = []
for MSISDN_NumberRanges in xml_ir21.findAll('GT_NumberRanges'):
    for MSISDN_RangeData in MSISDN_NumberRanges.findAll('RangeData'):
        for NumberRange in  MSISDN_RangeData.findAll('NumberRange'):
            nn_list.append([NumberRange.CC.text + NumberRange.NDC.text, 'nn'])
            
msrn_list = []
for MSISDN_NumberRanges in xml_ir21.findAll('MSRN_NumberRanges'):
    for NumberRange in  MSISDN_NumberRanges.findAll('NumberRange'):
        msrn_list.append([NumberRange.CC.text + NumberRange.NDC.text, 'msrn'])
        
e164_list = []
e164_list.extend(msisdn_list)
e164_list.extend(nn_list)
e164_list.extend(msrn_list)

for tadigs in xml_ir21.findAll('SenderTADIG'):
    tadig = tadigs.text
    print(tadig)
    
df = DataFrame(e164_list, columns = ['e164', 'type'])
df['tadig'] = tadig

display(df[df.type == 'nn'])

pd.merge(df_np.loc[(df_np.Operator == tadig) & (df_np.SSN == 7), ['Operator', 'Prefix', 'Number/Range', 'SSN']],
         df[df.type == 'nn'], how='left', left_on=['Operator', 'Prefix'], right_on=['tadig', 'e164'])

CRICL


,e164,type,tadig
0,5067000,nn,CRICL


,Operator,Prefix,Number/Range,SSN,e164,type,tadig
0,CRICL,5067000,1,7,5067000,nn,CRICL


In [ ]:
def fedine_nn_range(df):
    for GT_NN_START, GT_NN_END in zip(df.GT_NN_START.values, df.GT_NN_END.values):
        print(all([dig in ['1', '0'] for dig in str(int(GT_NN_END) - int(GT_NN_START) + 1)]))
        if all([dig in ['1', '0'] for dig in str(int(GT_NN_END) - int(GT_NN_START) + 1)]):
            gt = ''
            for digits in zip(GT_NN_START, GT_NN_END):
                if len(set(digits)) == 1:
                    print(GT_NN_START)
#     print(df.GT_NN_START.values.astype('str'))

fedine_nn_range(df_nn[df_nn.TADIG=='ARGTP'])

In [14]:
df_np[(df_np.Operator == tadig)].sort_values(by=['Number/Range','SSN','Prefix'])

NameError: name 'df_np' is not defined

In [175]:
def prepare_toc_create_netpfx(df):
    """df[['Prefix','Number/Range','SSN']] from NETPFX DMI table"""
    for [prefix, ssn] in df.loc[df.SSN != 1, ['Prefix','SSN']].values:
        print(f'dmi.netpfx.create,MCC="{}",MNC="{}",OPERATOR_REF="{}",PREFIX="{}",RANGE="1",SSN="{}";')

prepare_toc_delete_netpfx(df_np[(df_np.Operator == tadig)].sort_values(by=['Number/Range','SSN','Prefix']))

dmi.netpfx.remove,PREFIX="2250598786",SSN="149";
dmi.netpfx.remove,PREFIX="2250598787",SSN="149";
dmi.netpfx.remove,PREFIX="22505",SSN="7";
dmi.netpfx.remove,PREFIX="22505",SSN="149";


In [78]:
help dmi.netpfx.create,MCC="612",MNC="05",OPERATOR_REF="CIVTL",PREFIX="22505987",RANGE="1",SSN="7";

In [66]:
import socket
import socket
from os.path import join
host = "192.168.1.63"
port = 9999
# downloads = 'C:/Users/balob/Downloads'
file_name = 'IR21_SWETR_Telia Company AB_20191029102821.pdf'
s = socket.socket()
s.connect((host, port))
f = open(join(downloads, file_name), "rb")
l = f.read(1024)
while (l):
    s.send(l)
    l = f.read(1024)
f.close()
s.close()